# Part 1:

## 1- Importing the necessary libraries

In [1]:
import pandas as pd
import numpy as np

## 2- Reading the HTML table from the population website to a pandas dataframe

In [2]:
url_of_website = 'https://www.citypopulation.de/en/egypt/greatercairo/'
column_names = ['Governorate', 'Neighborhood', 'Population'] 

# Initiating the dataframe
Cairo_data_df = pd.DataFrame(columns=column_names)

# Reading the HTML table from the website
Cairo_data = pd.read_html(url_of_website)
Cairo_data[0].drop(['Native','Status','PopulationCensus1996-11-19','PopulationCensus2006-11-11'],axis=1 , inplace=True)
Cairo_data[0]
Cairo_data_df['Governorate'] = Cairo_data[0]['Governorate']
Cairo_data_df['Neighborhood'] = Cairo_data[0]['Name']
Cairo_data_df['Population'] = Cairo_data[0]['PopulationEstimate2019-07-01']
Cairo_data_df_new = Cairo_data_df.groupby(['Governorate','Neighborhood']).sum()
Cairo_data_df_new.head()

Population
Governorate Neighborhood                         
Al-Jīzah    Ad-Duqqī                        73309
            Al-'Ajūzah                     287818
            Al-'Umrāniyah                  378333
            Al-Ahrām                       681383
            Al-Badrashayn [Badrshein]      563294

## 3- Getting the neighborhoods data by using Foursquare

In [3]:
CLIENT_ID = 'YFPJU4FUJISRJZFAC3CYM0XVD4SWBYMMLJYHRYDNJTFMPFV0' # your Foursquare ID

CLIENT_SECRET = 'AXXB3G4UGM23JQGEBAXBQZHDQORPJBRN1EQJIAWNCI2XNATU' # your Foursquare Secret

VERSION = '20180605' # Foursquare API version

print('My credentails: ***')

print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails: ***
CLIENT_ID: YFPJU4FUJISRJZFAC3CYM0XVD4SWBYMMLJYHRYDNJTFMPFV0
CLIENT_SECRET:AXXB3G4UGM23JQGEBAXBQZHDQORPJBRN1EQJIAWNCI2XNATU


In [4]:
from geopy.geocoders import Nominatim
address='Cairo, Egypt'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Cairo, Egypt is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Cairo, Egypt is 30.048819, 31.243666.


In [5]:
neighborhood_lat=latitude
neighborhood_lon=longitude
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID, 
CLIENT_SECRET, 
VERSION, 
neighborhood_lat, 
neighborhood_lon, 
radius, 
LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=YFPJU4FUJISRJZFAC3CYM0XVD4SWBYMMLJYHRYDNJTFMPFV0&client_secret=AXXB3G4UGM23JQGEBAXBQZHDQORPJBRN1EQJIAWNCI2XNATU&v=20180605&ll=30.048819,31.243666&radius=500&limit=100'

In [6]:
import json
from pandas.io.json import json_normalize
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e68ebd547e0d6001c00b274'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Wust El-Balad',
  'headerFullLocation': 'Wust El-Balad, Cairo',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 48,
  'suggestedBounds': {'ne': {'lat': 30.053319004500008,
    'lng': 31.24885501032473},
   'sw': {'lat': 30.044318995499996, 'lng': 31.238476989675274}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53d928bd498ea9f619d7fa4f',
       'name': 'Halawa (حلاوه)',
       'location': {'address': '165 Mohamed Farid St',
        'crossStreet': 'Abd El Khalek Tharwat St',
        'lat': 30.050450186118397,
        'lng': 31.244201225229514,
       

In [7]:
results['response']['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '53d928bd498ea9f619d7fa4f',
   'name': 'Halawa (حلاوه)',
   'location': {'address': '165 Mohamed Farid St',
    'crossStreet': 'Abd El Khalek Tharwat St',
    'lat': 30.050450186118397,
    'lng': 31.244201225229514,
    'labeledLatLngs': [{'label': 'display',
      'lat': 30.050450186118397,
      'lng': 31.244201225229514}],
    'distance': 188,
    'cc': 'EG',
    'city': 'وسط البلد',
    'state': 'القاهرة',
    'country': 'مصر',
    'formattedAddress': ['165 Mohamed Farid St (Abd El Khalek Tharwat St)',
     'وسط البلد',
     'القاهرة',
     'مصر']},
   'categories': [{'id': '4bf58dd8d48988d103951735',
     'name': 'Clothing Store',
     'pluralName': 'Clothing Stores',
     'shortName': 'Apparel',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/apparel_',
      'suffix': '.png'},
     'primary': 

In [8]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### A function which specifies if a certain venue is popular or not

In [9]:
# function that tells is the Neighborhood is popular
# Needs Modification
def is_popular(row):
    try:
        popular_list = row['response']['groups'][0]['items'][0]['reasons']['items'][0]['summary']
    except:
        print('There is no information about the popularity of this place')
        
    if len(popular_list) == 0:
        return None
    else:
        return popular_list

## 4- Getting the data ready

In [10]:
venues = results['response']['groups'][0]['items']
venues
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Halawa (حلاوه),Clothing Store,30.050450,31.244201
1,Simonds,Coffee Shop,30.049971,31.242260
2,Happy City Roof Bar,Hotel Bar,30.046346,31.243932
3,Dar El Maaref (دار المعارف),Bookstore,30.050467,31.242328
4,El Abd (العبد),Pastry Shop,30.049617,31.239840
